## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ecija,37.5422,-5.0826,54.75,overcast clouds,86,95,3.42,ES,2022-02-13 01:41:24
1,1,Shellbrook,53.2168,-106.4011,-2.97,broken clouds,99,60,6.40,CA,2022-02-13 01:41:25
2,2,Ribeira Grande,38.5167,-28.7000,61.36,overcast clouds,93,100,15.19,PT,2022-02-13 01:41:25
3,3,Hilo,19.7297,-155.0900,80.33,clear sky,70,0,9.22,US,2022-02-13 01:36:49
4,4,Lompoc,34.6391,-120.4579,77.14,clear sky,44,0,5.75,US,2022-02-13 01:41:26


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Hilo,19.7297,-155.0900,80.33,clear sky,70,0,9.22,US,2022-02-13 01:36:49
4,4,Lompoc,34.6391,-120.4579,77.14,clear sky,44,0,5.75,US,2022-02-13 01:41:26
7,7,Sao Filipe,14.8961,-24.4956,77.41,overcast clouds,48,100,18.99,CV,2022-02-13 01:41:28
11,11,Atuona,-9.8000,-139.0333,77.61,light rain,77,15,12.41,PF,2022-02-13 01:41:29
17,17,Cravo Norte,6.3017,-70.2041,83.16,overcast clouds,48,100,5.64,CO,2022-02-13 01:41:32
19,19,Vaini,-21.2000,-175.2000,87.96,few clouds,79,20,10.36,TO,2022-02-13 01:41:33
20,20,Avarua,-21.2078,-159.7750,82.45,broken clouds,69,75,2.30,CK,2022-02-13 01:41:33
21,21,Rikitea,-23.1203,-134.9692,78.96,clear sky,71,0,16.06,PF,2022-02-13 01:41:33
29,29,New Norfolk,-42.7826,147.0587,81.66,overcast clouds,26,100,6.87,AU,2022-02-13 01:41:38
31,31,Luwuk,-0.9516,122.7875,81.43,overcast clouds,78,100,2.39,ID,2022-02-13 01:41:39


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                175
City                   175
Lat                    175
Lng                    175
Max Temp               175
Current Description    175
Humidity               175
Cloudiness             175
Wind Speed             175
Country                175
Date                   175
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country","Current Description", "Max Temp", "Humidity","Cloudiness","Wind Speed","Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Hotel Name
3,Hilo,US,clear sky,80.33,70,0,9.22,19.7297,-155.0900,
4,Lompoc,US,clear sky,77.14,44,0,5.75,34.6391,-120.4579,
7,Sao Filipe,CV,overcast clouds,77.41,48,100,18.99,14.8961,-24.4956,
11,Atuona,PF,light rain,77.61,77,15,12.41,-9.8000,-139.0333,
17,Cravo Norte,CO,overcast clouds,83.16,48,100,5.64,6.3017,-70.2041,
19,Vaini,TO,few clouds,87.96,79,20,10.36,-21.2000,-175.2000,
20,Avarua,CK,broken clouds,82.45,69,75,2.30,-21.2078,-159.7750,
21,Rikitea,PF,clear sky,78.96,71,0,16.06,-23.1203,-134.9692,
29,New Norfolk,AU,overcast clouds,81.66,26,100,6.87,-42.7826,147.0587,
31,Luwuk,ID,overcast clouds,81.43,78,100,2.39,-0.9516,122.7875,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
clean_hotel_df.head()

,City,Country,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Hotel Name
3,Hilo,US,clear sky,80.33,70,0,9.22,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Lompoc,US,clear sky,77.14,44,0,5.75,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
7,Sao Filipe,CV,overcast clouds,77.41,48,100,18.99,14.8961,-24.4956,Hotel Xaguate
11,Atuona,PF,light rain,77.61,77,15,12.41,-9.8000,-139.0333,Villa Enata
17,Cravo Norte,CO,overcast clouds,83.16,48,100,5.64,6.3017,-70.2041,Hotel Potosí Los Mirtos


In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))